### ドライブのマウント

In [1]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ライブラリ・モジュールのインポート

In [2]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00


### シード値の設定

In [3]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [4]:
# 平均値と標準偏差を計算するためのCIFAR10データセットの前処理を定義
pre_transform_cifar10 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR10データセットの読み込み
pre_train_dataset_cifar10 = datasets.CIFAR10(root='/content/data/', download=True, transform=pre_transform_cifar10)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar10 = 0.0
pre_std_cifar10 = 0.0
pre_total_samples_cifar10 = len(pre_train_dataset_cifar10)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar10:
    pre_image, _ = data
    pre_mean_cifar10 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar10 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar10 /= pre_total_samples_cifar10
pre_std_cifar10 /= pre_total_samples_cifar10

print("データセット全体の平均値: ", pre_mean_cifar10)
print("データセット全体の標準偏差: ", pre_std_cifar10)

100%|██████████| 170498071/170498071 [00:13<00:00, 12888437.96it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data/
データセット全体の平均値:  tensor([0.4914, 0.4822, 0.4465])
データセット全体の標準偏差:  tensor([0.2023, 0.1994, 0.2010])


In [5]:
# 学習用のCIFAR10データセットの前処理を定義
train_transform_cifar10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])
# テスト用のCIFAR10データセットの前処理を定義
test_transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar10, std=pre_std_cifar10)
    ])

# 学習用のCIFAR10データセットの読み込み
train_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=True, transform=train_transform_cifar10, download=True)
# テスト用のCIFAR10データセットの読み込み
test_dataset_cifar10 = torchvision.datasets.CIFAR10(root='/content/data/', train=False, transform=test_transform_cifar10, download=True)

# 学習用のCIFAR10データローダーを作成
train_loader_cifar10 = torch.utils.data.DataLoader(dataset=train_dataset_cifar10, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR10データローダーを作成
test_loader_cifar10 = torch.utils.data.DataLoader(dataset=test_dataset_cifar10, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [6]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [7]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [8]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [9]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [10]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBuildingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.bn1 = NonAffineBatchNorm(in_channels)
        self.conv1 = supermaskconv3x3(in_channels, out_channels, stride)
        self.bn2 = NonAffineBatchNorm(out_channels)
        self.conv2 = supermaskconv3x3(out_channels, out_channels)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != out_channels or stride > 1:
            self.shortcut = supermaskconv1x1(in_channels, out_channels, stride)
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        # 残差写像と恒等写像の要素毎の和を計算
        out += self.shortcut(x)
        return out

In [11]:
class SupermaskWideResNet(nn.Module):
    def __init__(self, block, depth, k, num_classes=10):
        super().__init__()
        assert (depth - 4) % 6 == 0, "depth should be 6n + 4"
        n = (depth - 4) // 6
        channels = [16, 16 * k, 32 * k, 64 * k]
        self.conv1 = supermaskconv3x3(3, channels[0])
        # Residual Blocks（1)
        self.layer1 = self._make_layer(block, channels[0], channels[1], n)
        # Residual Blocks（2）
        self.layer2 = self._make_layer(block, channels[1], channels[2], n, stride=2)
        # Residual Blocks（3）
        self.layer3 = self._make_layer(block, channels[2], channels[3], n, stride=2)
        self.bn = NonAffineBatchNorm(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channels[3], num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        layers = []
        # 最初の Residual Block（stride=stride）
        layers.append(block(in_channels, out_channels, stride))
        # 残りの Residual Block（stride=1）
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [12]:
# モデルを呼び出す関数を定義
def supermaskwideresnet28_10_10():
    return SupermaskWideResNet(SupermaskBuildingBlock, 28, 10, num_classes=10)

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [14]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.0 %, Loss: 2.3030
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.0 %, Loss: 2.3019
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.0 %, Loss: 2.3034
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.0 %, Loss: 2.3027
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 9.964 %, Loss: 2.3023
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 10.0 %, Loss: 2.3035
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 10.018 %, Loss: 2.3029
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 9.726 %, Loss: 2.3027
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 9.818 %, Loss: 2.3025
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 9.906 %, Loss: 2.3028
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 9.83 %, Loss: 2.3

In [15]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [16]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 10.0 %


In [17]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [18]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 26.298 %, Loss: 1.7769
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 32.308 %, Loss: 1.7014
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 36.36 %, Loss: 1.6170
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 44.656 %, Loss: 1.7380
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 51.732 %, Loss: 1.2196
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 56.586 %, Loss: 1.2021
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 59.458 %, Loss: 1.1469
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 62.034 %, Loss: 1.0205
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 65.718 %, Loss: 0.9186
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 67.978 %, Loss: 0.8690
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 69.4

In [19]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [20]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 82.46 %


In [21]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [ ]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.956 %, Loss: 1.6557
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 39.98 %, Loss: 1.6699
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 47.936 %, Loss: 1.3625
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 54.316 %, Loss: 1.2506
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 59.122 %, Loss: 1.0801
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 62.648 %, Loss: 0.9995
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 66.474 %, Loss: 0.8189
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 69.658 %, Loss: 0.8413
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 71.102 %, Loss: 0.8621
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 72.638 %, Loss: 0.7657
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 72.6

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 87.75 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [ ]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 37.982 %, Loss: 1.3115
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 52.692 %, Loss: 1.2023
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 59.21 %, Loss: 0.9385
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 64.812 %, Loss: 0.8086
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 69.222 %, Loss: 0.8579
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 72.426 %, Loss: 0.8384
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 74.524 %, Loss: 0.6606
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 76.296 %, Loss: 0.6295
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 77.48 %, Loss: 0.6173
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 78.422 %, Loss: 0.5623
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 79.45

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 89.03 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [ ]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 37.972 %, Loss: 1.4386
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 53.372 %, Loss: 1.1589
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 58.766 %, Loss: 1.0361
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 65.816 %, Loss: 0.8151
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 70.128 %, Loss: 0.8735
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 73.6 %, Loss: 0.7518
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 76.01 %, Loss: 0.6498
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 76.674 %, Loss: 0.6694
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 78.132 %, Loss: 0.4920
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 80.228 %, Loss: 0.6034
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 81.12 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.32 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [ ]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 38.908 %, Loss: 1.3540
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 54.398 %, Loss: 1.1069
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 60.946 %, Loss: 1.0198
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 67.364 %, Loss: 0.8576
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 72.094 %, Loss: 0.7232
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 75.918 %, Loss: 0.6224
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 77.9 %, Loss: 0.5747
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 79.574 %, Loss: 0.6008
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 80.366 %, Loss: 0.5836
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.498 %, Loss: 0.5519
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 82.28

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.36 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [ ]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 39.218 %, Loss: 1.4676
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 55.45 %, Loss: 1.0965
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 61.156 %, Loss: 0.8718
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 66.524 %, Loss: 0.8202
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 72.244 %, Loss: 0.6960
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 75.29 %, Loss: 0.6909
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 77.602 %, Loss: 0.6741
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 78.894 %, Loss: 0.5400
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 80.128 %, Loss: 0.6435
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.094 %, Loss: 0.4418
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 81.71

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.6 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [22]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 22.902 %, Loss: 1.9725
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 29.84 %, Loss: 1.8799
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 32.41 %, Loss: 1.8628
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 27.856 %, Loss: 1.9422
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 31.828 %, Loss: 1.8443
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 34.934 %, Loss: 1.9135
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 37.792 %, Loss: 1.5574
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 41.68 %, Loss: 1.4507
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 44.934 %, Loss: 1.4999
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 47.296 %, Loss: 1.4666
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 48.508

In [23]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [24]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 60.17 %


In [25]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [26]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 27.844 %, Loss: 1.7288
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 36.636 %, Loss: 1.7092
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 38.358 %, Loss: 1.9254
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 40.236 %, Loss: 1.5300
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 45.732 %, Loss: 1.9478
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 42.942 %, Loss: 1.4392
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 53.774 %, Loss: 1.2507
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 58.684 %, Loss: 1.1073
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 61.878 %, Loss: 0.9819
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 64.35 %, Loss: 1.0623
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 65.4

In [27]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [28]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 83.02 %


In [29]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [ ]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.374 %, Loss: 1.7115
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 43.162 %, Loss: 1.4378
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 49.508 %, Loss: 1.2814
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 53.06 %, Loss: 1.1748
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 58.984 %, Loss: 1.1403
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 63.638 %, Loss: 0.9349
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 65.97 %, Loss: 0.8392
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 68.724 %, Loss: 0.7777
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 69.3 %, Loss: 0.8599
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 71.364 %, Loss: 0.7869
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 72.656 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 88.91 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [ ]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 37.402 %, Loss: 1.4884
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 51.158 %, Loss: 1.1588
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 57.904 %, Loss: 1.0572
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 64.406 %, Loss: 1.0607
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 68.716 %, Loss: 0.7588
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 72.15 %, Loss: 0.8505
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 73.93 %, Loss: 0.7415
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 74.99 %, Loss: 0.7184
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 77.098 %, Loss: 0.6433
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 78.14 %, Loss: 0.6266
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 78.972 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 90.26 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [ ]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 37.742 %, Loss: 1.4419
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 52.342 %, Loss: 1.1793
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 57.862 %, Loss: 1.0869
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 65.03 %, Loss: 0.8775
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 69.702 %, Loss: 0.6411
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 73.062 %, Loss: 0.8062
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 75.212 %, Loss: 0.6601
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 77.052 %, Loss: 0.7113
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 77.904 %, Loss: 0.6495
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 79.304 %, Loss: 0.6340
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 80.0

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 92.89 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [ ]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 38.774 %, Loss: 1.4864
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 53.538 %, Loss: 1.0992
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 59.836 %, Loss: 1.0095
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 66.048 %, Loss: 0.8724
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 71.314 %, Loss: 0.7115
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 74.582 %, Loss: 0.6177
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 77.196 %, Loss: 0.6200
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 78.922 %, Loss: 0.5527
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 80.228 %, Loss: 0.5211
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.128 %, Loss: 0.4898
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 81.

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 92.96 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [ ]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 39.434 %, Loss: 1.3748
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 53.984 %, Loss: 1.2235
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 60.666 %, Loss: 1.0637
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 66.024 %, Loss: 0.9089
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 71.138 %, Loss: 0.8643
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 75.028 %, Loss: 0.6245
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 77.272 %, Loss: 0.6220
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 79.2 %, Loss: 0.5735
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 80.002 %, Loss: 0.5736
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.004 %, Loss: 0.4819
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 81.04

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 92.99 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [30]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.999_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 10.0 %, Loss: 2.3039
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.0 %, Loss: 2.3015
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.0 %, Loss: 2.3015
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.0 %, Loss: 2.3033
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 10.0 %, Loss: 2.3028
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 10.0 %, Loss: 2.3024
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 10.0 %, Loss: 2.3022
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 10.0 %, Loss: 2.3027
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 9.72 %, Loss: 2.3025
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 10.0 %, Loss: 2.3026
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 9.72 %, Loss: 2.3026
Ep

In [31]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [32]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 10.0 %


In [33]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [34]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.995_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 30.588 %, Loss: 1.5714
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 43.056 %, Loss: 1.3413
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 47.884 %, Loss: 1.3984
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 53.24 %, Loss: 1.2188
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 57.468 %, Loss: 1.0579
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 58.284 %, Loss: 1.2867
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 60.598 %, Loss: 1.0337
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 64.896 %, Loss: 1.0309
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 64.134 %, Loss: 0.8930
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 66.76 %, Loss: 0.9391
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 67.27

In [35]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [36]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 85.55 %


In [37]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [ ]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.99_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 31.612 %, Loss: 1.6238
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 44.142 %, Loss: 1.3949
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 52.72 %, Loss: 1.1548
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 58.238 %, Loss: 1.0342
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 63.774 %, Loss: 0.9271
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 67.726 %, Loss: 0.7503
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 70.294 %, Loss: 0.7698
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 72.644 %, Loss: 0.9094
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 73.014 %, Loss: 0.9135
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 72.8 %, Loss: 0.9014
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 75.544

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 90.73 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [ ]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.95_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 36.848 %, Loss: 1.3684
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 52.714 %, Loss: 1.1457
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 60.66 %, Loss: 0.8519
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 66.064 %, Loss: 1.0205
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 70.414 %, Loss: 0.7526
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 73.65 %, Loss: 0.6519
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 75.534 %, Loss: 0.6841
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 77.33 %, Loss: 0.7071
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 78.456 %, Loss: 0.5036
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 79.64 %, Loss: 0.4487
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 80.636 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 92.9 %


In [19]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [ ]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.9_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 38.626 %, Loss: 1.3128
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 54.604 %, Loss: 1.1049
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 59.952 %, Loss: 1.1857
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 63.19 %, Loss: 0.8889
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 69.338 %, Loss: 0.8379
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 73.584 %, Loss: 0.6307
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 75.576 %, Loss: 0.6341
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 77.894 %, Loss: 0.6432
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 78.894 %, Loss: 0.6380
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 79.872 %, Loss: 0.5203
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 80.8

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.72 %


In [22]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [ ]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.7_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 40.194 %, Loss: 1.3868
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 55.538 %, Loss: 1.1154
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 62.47 %, Loss: 1.0367
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 68.15 %, Loss: 0.8284
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 73.048 %, Loss: 0.6396
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 75.746 %, Loss: 0.7685
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 77.85 %, Loss: 0.6028
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 80.128 %, Loss: 0.5041
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 81.29 %, Loss: 0.5693
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.802 %, Loss: 0.5286
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 82.964 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.88 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [ ]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskwideresnet28_10_10().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar10):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskWideResNet28_10_CIFAR10_100epochs_pr0.5_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 39.84 %, Loss: 1.5123
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 54.782 %, Loss: 1.1302
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 61.492 %, Loss: 0.8923
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 66.492 %, Loss: 0.8967
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 71.866 %, Loss: 0.7815
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 75.674 %, Loss: 0.7189
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 78.052 %, Loss: 0.6250
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 79.674 %, Loss: 0.6180
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 80.6 %, Loss: 0.6525
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 81.382 %, Loss: 0.5878
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 81.75 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar10:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 93.69 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()